In [32]:
#Import needed packages
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
import folium

In [33]:
#Download the data and change into df
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Data downloaded!


In [34]:
neighborhoods['Borough'].unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'],
      dtype=object)

In [35]:
SI = neighborhoods[neighborhoods['Borough'] == 'Manhattan']
print(SI.head())

       Borough        Neighborhood   Latitude  Longitude
6    Manhattan         Marble Hill  40.876551 -73.910660
100  Manhattan           Chinatown  40.715618 -73.994279
101  Manhattan  Washington Heights  40.851903 -73.936900
102  Manhattan              Inwood  40.867684 -73.921210
103  Manhattan    Hamilton Heights  40.823604 -73.949688


In [36]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# create map of Manhattan using latitude and longitude values
map_SI = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(SI['Latitude'], SI['Longitude'], SI['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SI)  
    
map_SI

In [37]:
print(SI['Neighborhood'].unique())
print(SI['Neighborhood'].count())

['Marble Hill' 'Chinatown' 'Washington Heights' 'Inwood'
 'Hamilton Heights' 'Manhattanville' 'Central Harlem' 'East Harlem'
 'Upper East Side' 'Yorkville' 'Lenox Hill' 'Roosevelt Island'
 'Upper West Side' 'Lincoln Square' 'Clinton' 'Midtown' 'Murray Hill'
 'Chelsea' 'Greenwich Village' 'East Village' 'Lower East Side' 'Tribeca'
 'Little Italy' 'Soho' 'West Village' 'Manhattan Valley'
 'Morningside Heights' 'Gramercy' 'Battery Park City' 'Financial District'
 'Carnegie Hill' 'Noho' 'Civic Center' 'Midtown South' 'Sutton Place'
 'Turtle Bay' 'Tudor City' 'Stuyvesant Town' 'Flatiron' 'Hudson Yards']
40


In [38]:
SI_Manhattan = SI[SI['Neighborhood'].str.contains('Manhattan')].reset_index(drop=True)
SI_Manhattan

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Manhattanville,40.816934,-73.957385
1,Manhattan,Manhattan Valley,40.797307,-73.964286


In [39]:
CLIENT_ID = 'YHNTSI3GJBFBI00VEAD44W0NRVKRMX2H0NWXNCD3M30YXMSC' # your Foursquare ID
CLIENT_SECRET = 'CTC2VKKQBAIP1NWTJ2422U1X51XNZOEMTZ3JG2OFEP3ELAWD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [40]:
neighborhood_latitude = SI_Manhattan.loc[0, 'Latitude']
neighborhood_longitude = SI_Manhattan.loc[0, 'Longitude']
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=YHNTSI3GJBFBI00VEAD44W0NRVKRMX2H0NWXNCD3M30YXMSC&client_secret=CTC2VKKQBAIP1NWTJ2422U1X51XNZOEMTZ3JG2OFEP3ELAWD&v=20180605&ll=40.8169344294978,-73.9573853935188&radius=1000&limit=100'

In [41]:
results = requests.get(url).json()

In [42]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Oasis Juice Bar,Juice Bar,40.815017,-73.958879
1,Dinosaur Bar-B-Que,BBQ Joint,40.818147,-73.960674
2,West Harlem Piers Park,Park,40.818793,-73.961029
3,Jin Ramen,Ramen Restaurant,40.815406,-73.958547
4,Hudson River Greenway Bike Path,Bike Trail,40.819282,-73.960733


In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['id'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue ID',
                  'Venue Category']
    
    return(nearby_venues)
Manhattan_venues = getNearbyVenues(names=SI_Manhattan['Neighborhood'],
                                   latitudes=SI_Manhattan['Latitude'],
                                   longitudes=SI_Manhattan['Longitude']
                                  )

Manhattanville
Manhattan Valley


In [45]:
Manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue ID,Venue Category
Neighborhood,,,,,,,
Manhattan Valley,43,43,43,43,43,43,43
Manhattanville,42,42,42,42,42,42,42


In [46]:
Manhattan_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue ID,Venue Category
0,Manhattanville,40.816934,-73.957385,Oasis Juice Bar,40.815017,-73.958879,50a64d65e4b0c753d6ab47a3,Juice Bar
1,Manhattanville,40.816934,-73.957385,Jin Ramen,40.815406,-73.958547,4f29cf06e4b022d4180801f3,Ramen Restaurant
2,Manhattanville,40.816934,-73.957385,Steep Rock West,40.816668,-73.957969,59389a503af98877bbb8dc6a,Climbing Gym
3,Manhattanville,40.816934,-73.957385,Dinosaur Bar-B-Que,40.818147,-73.960674,41b24f80f964a520661e1fe3,BBQ Joint
4,Manhattanville,40.816934,-73.957385,Go! Go! Curry!,40.815473,-73.958234,57a4cbac498ee795c5d0b864,Japanese Curry Restaurant
5,Manhattanville,40.816934,-73.957385,The Expat,40.815355,-73.960178,5d06e2eb59b5a40023f1c546,Gastropub
6,Manhattanville,40.816934,-73.957385,Kissaten Jin,40.815457,-73.958469,5320789f498e6416b2b6f814,Café
7,Manhattanville,40.816934,-73.957385,West Harlem Piers Park,40.818793,-73.961029,4ee678ffa17ceecb16341fc3,Park
8,Manhattanville,40.816934,-73.957385,The World Famous Cotton Club,40.817587,-73.960274,3fd66200f964a5206be31ee3,Lounge
9,Manhattanville,40.816934,-73.957385,Hudson River Greenway Bike Path,40.819282,-73.960733,4eb3e2550aaf1abede9a5312,Bike Trail


In [47]:
# one hot encoding
Manhattan_onehot = pd.get_dummies(Manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Manhattan_onehot['Neighborhood'] = Manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Manhattan_onehot.columns[-1]] + list(Manhattan_onehot.columns[:-1])
Manhattan_onehot = Manhattan_onehot[fixed_columns]

Manhattan_onehot.head()

,Neighborhood,American Restaurant,Arts & Crafts Store,BBQ Joint,Bakery,Bank,Bar,Bike Trail,Boutique,Bubble Tea Shop,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Climbing Gym,Clothing Store,Coffee Shop,Cosmetics Shop,Cuban Restaurant,Deli / Bodega,Diner,Dog Run,Dumpling Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Food & Drink Shop,Fried Chicken Joint,Gastropub,Grocery Store,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Hostel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Mexican Restaurant,Park,Peruvian Restaurant,Pizza Place,Playground,Ramen Restaurant,Scenic Lookout,Seafood Restaurant,Spanish Restaurant,Supermarket,Sushi Restaurant,Thai Restaurant,Vietnamese Restaurant,Wine Shop,Wings Joint,Yoga Studio
0,Manhattanville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Manhattanville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,Manhattanville,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Manhattanville,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Manhattanville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
Manhattan_grouped = Manhattan_onehot.groupby('Neighborhood').mean().reset_index()
Manhattan_grouped

,Neighborhood,American Restaurant,Arts & Crafts Store,BBQ Joint,Bakery,Bank,Bar,Bike Trail,Boutique,Bubble Tea Shop,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Climbing Gym,Clothing Store,Coffee Shop,Cosmetics Shop,Cuban Restaurant,Deli / Bodega,Diner,Dog Run,Dumpling Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Food & Drink Shop,Fried Chicken Joint,Gastropub,Grocery Store,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Hostel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Mexican Restaurant,Park,Peruvian Restaurant,Pizza Place,Playground,Ramen Restaurant,Scenic Lookout,Seafood Restaurant,Spanish Restaurant,Supermarket,Sushi Restaurant,Thai Restaurant,Vietnamese Restaurant,Wine Shop,Wings Joint,Yoga Studio
0,Manhattan Valley,0.00000,0.023256,0.00000,0.023256,0.00000,0.069767,0.00000,0.00000,0.023256,0.00000,0.023256,0.023256,0.023256,0.00000,0.023256,0.093023,0.00000,0.00000,0.000000,0.00000,0.023256,0.00000,0.023256,0.00000,0.023256,0.00000,0.023256,0.00000,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.00000,0.023256,0.00000,0.023256,0.023256,0.00000,0.00000,0.069767,0.023256,0.023256,0.046512,0.023256,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.023256,0.023256,0.023256,0.023256,0.046512
1,Manhattanville,0.02381,0.000000,0.02381,0.000000,0.02381,0.023810,0.02381,0.02381,0.000000,0.02381,0.023810,0.000000,0.023810,0.02381,0.000000,0.095238,0.02381,0.02381,0.047619,0.02381,0.000000,0.02381,0.000000,0.02381,0.000000,0.02381,0.023810,0.02381,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.047619,0.02381,0.000000,0.02381,0.000000,0.000000,0.02381,0.02381,0.047619,0.023810,0.000000,0.000000,0.000000,0.02381,0.02381,0.047619,0.02381,0.02381,0.02381,0.000000,0.000000,0.000000,0.000000,0.000000


In [50]:
num_top_venues = 5

for hood in Manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Manhattan_grouped[Manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Manhattan Valley----
                venue  freq
0         Coffee Shop  0.09
1                 Bar  0.07
2  Mexican Restaurant  0.07
3         Yoga Studio  0.05
4         Pizza Place  0.05


----Manhattanville----
                venue  freq
0         Coffee Shop  0.10
1  Seafood Restaurant  0.05
2  Mexican Restaurant  0.05
3       Deli / Bodega  0.05
4  Italian Restaurant  0.05




In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Manhattan_grouped['Neighborhood']

for ind in np.arange(Manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan Valley,Coffee Shop,Bar,Mexican Restaurant,Yoga Studio,Pizza Place,Ethiopian Restaurant,Hostel,Health Food Store,Hawaiian Restaurant,Wings Joint
1,Manhattanville,Coffee Shop,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Deli / Bodega,American Restaurant,Climbing Gym,Japanese Curry Restaurant,Indian Restaurant,Café


In [58]:
# set number of clusters
kclusters = 2

Manhattan_grouped_clustering = richmond_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1], dtype=int32)

In [59]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Manhattan_merged = SI_richmond

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Manhattan_merged = Manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Manhattanville,40.816934,-73.957385,1,Coffee Shop,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Deli / Bodega,American Restaurant,Climbing Gym,Japanese Curry Restaurant,Indian Restaurant,Café
1,Manhattan,Manhattan Valley,40.797307,-73.964286,0,Coffee Shop,Bar,Mexican Restaurant,Yoga Studio,Pizza Place,Ethiopian Restaurant,Hostel,Health Food Store,Hawaiian Restaurant,Wings Joint


In [60]:
Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 0, Manhattan_merged.columns[[1] + list(range(5, Manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Manhattan Valley,Coffee Shop,Bar,Mexican Restaurant,Yoga Studio,Pizza Place,Ethiopian Restaurant,Hostel,Health Food Store,Hawaiian Restaurant,Wings Joint


In [62]:
Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 1, Manhattan_merged.columns[[1] + list(range(5, Manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattanville,Coffee Shop,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Deli / Bodega,American Restaurant,Climbing Gym,Japanese Curry Restaurant,Indian Restaurant,Café


In [67]:
Manhattan_menu = Manhattan_venues[Manhattan_venues['Venue Category'].str.contains('Coffee Shop|Pizza Place|Fast Food Restaurant|Mexican Restaurant|Italian Restaurant')].reset_index(drop=True)
Manhattan_menu

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue ID,Venue Category
0,Manhattanville,40.816934,-73.957385,Pisticci Ristorante,40.814015,-73.960266,457f1183f964a5204b3f1fe3,Italian Restaurant
1,Manhattanville,40.816934,-73.957385,Plowshares Coffee,40.813705,-73.955776,5c58826ad1a402002c0b8793,Coffee Shop
2,Manhattanville,40.816934,-73.957385,El Porton,40.814399,-73.959330,4acfe332f964a520afd620e3,Mexican Restaurant
3,Manhattanville,40.816934,-73.957385,Bettolona,40.814084,-73.959574,4c956003f7cfa1cd2e2ebd15,Italian Restaurant
4,Manhattanville,40.816934,-73.957385,Chinelos,40.820190,-73.953769,4bd4e5076798ef3b92a6628d,Mexican Restaurant
5,Manhattanville,40.816934,-73.957385,Starbucks,40.815074,-73.958930,58982463469aef72f3579b25,Coffee Shop
6,Manhattanville,40.816934,-73.957385,Kuro Kuma,40.813892,-73.960027,4fef2f45e4b0562c835d6b7c,Coffee Shop
7,Manhattanville,40.816934,-73.957385,Dear Mama Coffee,40.817333,-73.959494,5b0345aa9ba3e5002c6bfeb8,Coffee Shop
8,Manhattan Valley,40.797307,-73.964286,Starbucks,40.795369,-73.965589,4fdc9a79e4b0735a6deafc25,Coffee Shop
9,Manhattan Valley,40.797307,-73.964286,Osteria 106,40.798825,-73.961793,5962a53dc824ae6362e9e625,Italian Restaurant


In [114]:
VENUE_ID = Manhattan_menu['Venue ID'].loc[0]
url = 'https://api.foursquare.com/v2/venues/{}/menu?&client_id={}&client_secret={}&v={}'.format(
            VENUE_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
            )
results = requests.get(url).json()['response']['menu']['menus']['items']

In [116]:
results[0]['entries']['count']

6

In [105]:
VENUE_ID = Manhattan_menu['Venue ID'].loc[1]
url = 'https://api.foursquare.com/v2/venues/{}/menu?&client_id={}&client_secret={}&v={}'.format(
            VENUE_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
            )
results = requests.get(url).json()['response']['menu']['menus']

In [106]:
results

{'count': 0}

In [ ]:
results[0]['entries']['count']

In [140]:
totalfinal = []

totalfinal.append([(results[0]['entries']['items'][i]['name'].split(':')[0] for i in range(0,2))])

totalf = pd.DataFrame([item for Total_ in totalfinal for item in Total_])
totalf.columns = ['Name','Content']

In [141]:
totalfinal

[[<generator object <genexpr> at 0x7fa601ae5db0>]]